
# prediccion de incidentes viales en la ciudad de Medellin

## importacion librerias necesarias

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


## lectura y exploracion de los datos

In [2]:
#lectura dataframe saltando las filas malas
df=pd.read_csv("incidentes_viales.csv",on_bad_lines='skip',index_col=False,sep=";",encoding="UTF-8")
print(df.head())

    AÑO  CBML CLASE_ACCIDENTE      DIRECCION       DIRECCION ENCASILLADA  \
0  2015  1013  Caida Ocupante    CR 46 CL 43    CR  046   043  000 00000   
1  2015  1020          Choque  CR 43 A CL 29  CR  043 A   029  000 00000   
2  2015  0205            Otro    CR 46 CL 37    CR  046   037  000 00000   
3  2015  0101       Atropello   CR 34 CL 112    CR  034   112  000 00000   
4  2015  1504          Choque    CR 50 CL 14    CR  050   014  000 00000   

         DISEÑO  EXPEDIENTE      FECHA_ACCIDENTE          FECHA_ACCIDENTES  \
0  Tramo de via  A000259731  21/10/2015 05:58:00  2015-10-21T10:58:00.000Z   
1  Tramo de via  A000261725  05/11/2015 03:00:00  2015-11-05T08:00:00.000Z   
2  Tramo de via  A000259739  21/10/2015 07:40:00  2015-10-21T12:40:00.000Z   
3  Tramo de via  A000259752  21/10/2015 12:00:00  2015-10-21T17:00:00.000Z   
4  Tramo de via  A000259949  03/11/2015 06:16:00  2015-11-03T11:16:00.000Z   

  GRAVEDAD_ACCIDENTE  MES NRO_RADICADO NUMCOMUNA                     BARRI

C:\Users\johan\anaconda3\envs\tensorflow_gpu\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#revisamos las columnas
print(df.columns)
print("numero de columnas iniciales ",len(df.columns))
print("numero de filas iniciales ",df.shape[0])


Index(['AÑO', 'CBML', 'CLASE_ACCIDENTE', 'DIRECCION', 'DIRECCION ENCASILLADA',
       'DISEÑO', 'EXPEDIENTE', 'FECHA_ACCIDENTE', 'FECHA_ACCIDENTES',
       'GRAVEDAD_ACCIDENTE', 'MES', 'NRO_RADICADO', 'NUMCOMUNA', 'BARRIO',
       'COMUNA', 'LOCATION', 'X', 'Y'],
      dtype='object')
numero de columnas iniciales  18
numero de filas iniciales  270765


## limpieza de datos

In [4]:
#eliminacion de datos con nulos
df.dropna(inplace=True)
#eliminar datos duplicados
df.drop_duplicates(inplace=True)
print(df.shape[0])

250686


In [5]:
#AÑO-Año de ocurrencia del incidente
df["AÑO"]=df["AÑO"].apply(lambda x : re.findall(r"[0-9]{4}",str(x))[0])
print(df["AÑO"].unique())

['2015' '2014' '2016' '2017' '2018' '2019' '2020']


In [6]:
#CBML-Codigo catastral que corresponde al codigo comuna, barrio, manzana, lote catastral de un predio.
print(len(df["CBML"].unique()))

521


In [7]:
#CLASE_ACCIDENTE-Clasificación del IPAT sobre la clase de accidente de transito: choque, atropello, volcamiento,
#caida de ocupante, incendio, u otro (que no corresponde a las anteriores 5 clasificaciones, p. ej: sumersión)
df["CLASE_ACCIDENTE"]=df["CLASE_ACCIDENTE"].apply(lambda x : str(x).replace("Caída de Ocupante","Caida Ocupante").lower())
print((df["CLASE_ACCIDENTE"].unique()))

['caida ocupante' 'choque' 'otro' 'atropello' 'volcamiento' 'incendio']


In [8]:
#DIRECCION-Dirección donde ocurrió el incidente
##aparentemente se encuentra ya formateada
print((df["DIRECCION"].sample(10)))

198658       CR 70 CL 48
221670    CR 64 C CL 103
118437     CL 49 CR 20 B
211911       CR 57 CL 58
132572       CR 80 CL 64
45167         CR 52 CL 5
176668       CL 75 CR 75
79330        CL 65 CR 83
259779     CR 53 A CL 42
160185       CR 52 CL 94
Name: DIRECCION, dtype: object


In [9]:
#DIRECCION ENCASILLADA-	Dirección encasillada que entrega el geocodificador
##aparentemente se encuentra ya formateada
##variable con poca informacion util dada la variable anterior--escoger una de las 2
print((df["DIRECCION ENCASILLADA"].sample(10)))

93208       CL  056   057  000 00000
102331     CR  049  S 014  000 00000
266430      CR  057   054  000 00000
233498      CR  052   080  000 00000
69021       CR  049   032  000 00000
152710      CL  081   050  000 00000
90077     CR  043 C   008  000 00000
139685    CR  064 C   095  000 00000
108783    CR  064 C   078  000 00000
1625      CR  040   050 B  000 00000
Name: DIRECCION ENCASILLADA, dtype: object


In [10]:
#DISENO-Sitio de la vía donde ocurrió el accidente: Cicloruta, Glorieta, Interseccion, Lote o Predio, Paso a Nivel, Paso Elevado,
#Paso Inferior, Pontón, Puente, Tramo de via, Tunel, Via peatonal
df["DISEÑO"]=df["DISEÑO"].apply(lambda x : str(x).replace("Pont\\xF3n","Pontón").lower())
print((df["DISEÑO"].unique()))

['tramo de via' 'interseccion' 'glorieta' 'lote o predio' 'paso a nivel'
 'puente' 'paso elevado' 'ciclo ruta' 'paso inferior' 'via peatonal'
 'pontón' 'tunel']


In [11]:
#EXPEDIENTE
#Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
##variable  aprarentemente con poco valor
print((df["EXPEDIENTE"].sample(10)))

103960    A000176660
194562    A000498889
197711    A000582314
25458     A000838481
243124    A000380156
79296     A000375184
223004    A000254004
200124    A000778689
246940    A000482137
129734    A000371633
Name: EXPEDIENTE, dtype: object


In [12]:
#FECHA_ACCIDENTE
#Fecha del accidente, proviene del IPAT - Informe Policial de accidente de Tránsito
df["FECHA_ACCIDENTE"] = pd.to_datetime(df["FECHA_ACCIDENTE"],dayfirst=True)
print((df["FECHA_ACCIDENTE"].sample(10)))

270251   2020-08-11 14:30:00
223532   2016-10-01 05:00:00
324      2015-10-23 04:00:00
9005     2017-08-09 15:30:00
221913   2015-07-16 09:30:00
127561   2016-03-18 13:20:00
76868    2015-03-12 06:50:00
253263   2019-04-25 18:00:00
170780   2019-10-24 14:00:00
231379   2016-02-27 10:10:00
Name: FECHA_ACCIDENTE, dtype: datetime64[ns]


In [13]:
#FECHA_ACCIDENTES
#Fecha de los accidente (formato YYYY-MM-DD hh:mi:ss), proviene del IPAT - Informe Policial de accidentes de Tránsito
##veriable redundante con la anterior
print((df["FECHA_ACCIDENTES"].head(10)))

0    2015-10-21T10:58:00.000Z
1    2015-11-05T08:00:00.000Z
2    2015-10-21T12:40:00.000Z
3    2015-10-21T17:00:00.000Z
4    2015-11-03T11:16:00.000Z
5    2015-11-05T09:30:00.000Z
6    2015-10-18T09:40:00.000Z
7    2015-11-04T09:46:00.000Z
8    2015-10-21T12:00:00.000Z
9    2015-11-05T12:00:00.000Z
Name: FECHA_ACCIDENTES, dtype: object


In [14]:
#GRAVEDAD_ACCIDENTE
#Clasificación del IPAT - Informe Policial de Accidentes de Tránsito, sobre la gravedad del accidente, corresponde al resultado 
#más grave presentado en el accidente. Daños materiales "Sólo daños", accidente con heridos "Herido", accidente con muertos 
#"Muerto". No indica cantidad
df["GRAVEDAD_ACCIDENTE"]=df["GRAVEDAD_ACCIDENTE"].apply(lambda x : str(x).replace("Solo da\\xF1os","Solo daños").lower())
print((df["GRAVEDAD_ACCIDENTE"].unique()))

['con heridos' 'solo daños' 'con muertos']


In [15]:
#MES
#Mes de ocurrencia del incidente vial
print((df["MES"].unique()))

[10 11 12  9  4  3  5  6  7  8  1  2]


In [16]:
#NRO_RADICADO
#Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento
#no tiene mucho valor aparentemente para nuestros fines
print((df["NRO_RADICADO"].head()))

0    1508668
1    1510621
2    1508691
3    1508694
4    1510634
Name: NRO_RADICADO, dtype: object


In [17]:
#NUMCOMUNA
#Numero de la comununa en la que ocurrio incidente vial
#se eliminan las filas con valores que no corespondan con 
df=df[df["NUMCOMUNA"]!="AU"]
df=df[df["NUMCOMUNA"]!="Sin Inf"]
df=df[df["NUMCOMUNA"]!="In"]
df=df[df["NUMCOMUNA"]!="SN"]
df=df[df["NUMCOMUNA"]!="0"]
#asuminos que las comunas 60,70,80,90 son las comunas 6,7,8,9 respectivamente
df["NUMCOMUNA"]=df["NUMCOMUNA"].apply(lambda x : (int(x)))
df["NUMCOMUNA"]=df["NUMCOMUNA"].apply(lambda x : x/10 if x >20 else x)
df["NUMCOMUNA"]=df["NUMCOMUNA"].apply(lambda x : str(int(x)))
print(sorted(df["NUMCOMUNA"].unique()))

['1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5', '6', '7', '8', '9']


In [18]:
#BARRIO
print(len(df["BARRIO"].unique()))
#Barrio de ocurrencia del incidente vial
#ó por o
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).lower().replace("\\xf3","o").replace("ó","o"))
#é por e 
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))
#á por a 
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))
#ñ
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xf1","ñ"))
#\\xed í por i
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xed","i").replace("í","i"))
#\\xfa ú por u
df["BARRIO"]=df["BARRIO"].apply(lambda x : str(x).replace("\\xfa","u").replace("ú","u"))
print((df["BARRIO"].unique()))

488
['barrio colon' 'san diego' 'la francia' 'santo domingo savio no. 1'
 'santa fe' 'la florida' 'los conquistadores' 'la alpujarra' 'villa nueva'
 'facultad de minas u. nacional' 'la esperanza' 'perpetuo socorro'
 'campo valdes no. 1' 'miraflores' 'oleoducto' 'terminal de transporte'
 'gerona' 'u.d. atanasio girardot' 'alejandro echavarria'
 'cabecera san antonio de prado' 'manila' 'los balsos no.2' 'moravia'
 'tenche' 'universidad nacional' 'los colores' 'caribe'
 'juan xxiii la quiebra' 'belalcazar' 'santa cruz' 'sucre' 'la candelaria'
 'el estadio' 'suramericana' 'la gloria' 'centro administrativo'
 'las estancias' 'aures no. 2' 'los angeles' 'castilla' 'la salle'
 'la america' 'picacho' 'bosques de san pablo' 'la aguacatala'
 'playon de los comuneros' 'estacion villa' 'la colina' 'popular'
 'villa del socorro' 'cucaracho' 'santa maria de los angeles'
 'el chagualo' 'la mansion' 'san benito' 'la castellana' 'jesus nazareno'
 'loreto' 'asomadera no. 1' 'el salvador' 'cristo rey' 'm

In [19]:
#COMUNA
#Denominación con la cual se identifica cada Comuna o Corregimiento. 01:Popular 02:Santa Cruz 03:Manrique 04:Aranjuez 05:Castilla 06:Doce de Octubre 07:Robledo 08:Villa Hermosa 09:Buenos Aires 10:La Candelaria 11:Laureles - Estadio 12:La América 13:San Javier 14:El Poblado 15:Guayabal 16:Belén 50:San Sebastián de Palmitas 60:San Cristobal 70:Altavista 80:San Antonio de Prado 90:Santa Elena 99:Toda la Ciudad
##variable redundante
#ó por o
df["COMUNA"]=df["COMUNA"].apply(lambda x : str(x).lower().replace("\\xf3","o").replace("ó","o"))
#á por a 
df["COMUNA"]=df["COMUNA"].apply(lambda x : str(x).replace("\\xc1","a").replace("\\xe1","a").replace("á","a"))
#é por e 
df["COMUNA"]=df["COMUNA"].apply(lambda x : str(x).replace("\\xe9","e").replace("é","e"))
print((df["COMUNA"].unique()))

['la candelaria' 'santa cruz' 'popular' 'guayabal' 'el poblado'
 'laureles estadio' 'robledo' 'doce de octubre' 'aranjuez' 'buenos aires'
 'castilla' 'corregimiento de san antonio de prado' 'san javier'
 'villa hermosa' 'belen' 'manrique' 'la america'
 'corregimiento de altavista' 'corregimiento de san cristobal'
 'corregimiento de santa elena' 'in'
 'corregimiento de san sebastian de palmitas' 'au']


In [20]:
#LOCATION
#Fuente de información con la cual se realizó la geocodificación
print((df["LOCATION"].head(10)))

0    [-75.5688011014, 6.24312304123]
1    [-75.5693883283, 6.22870030622]
2    [-75.5503911403, 6.29869502502]
3    [-75.5442195069, 6.30019383306]
4     [-75.578180998, 6.21699243701]
5      [-75.561118009, 6.2047775959]
6    [-75.5810144962, 6.24576066323]
7    [-75.5752597216, 6.24685341912]
8    [-75.5624178534, 6.25341454089]
9    [-75.5931430393, 6.27330476643]
Name: LOCATION, dtype: object


In [21]:
#X
#Coordenada X en metros del accidente, en sistema de coordenadas MAGNA Medellin Local
print(df["X"].head(10))

0    834949.69
1    834880.17
2    837004.94
3    837688.53
4    833903.11
5    835788.30
6    833598.49
7    834235.89
8    835659.53
9    832264.64
Name: X, dtype: float64


In [22]:
#Y
#Coordenada Y en metros del accidente, en sistema de coordenadas MAGNA Medellin Local
print(df["Y"].head(10))

0    1182357.34
1    1180762.02
2    1188499.17
3    1188663.05
4    1179469.60
5    1178113.02
6    1182652.97
7    1182772.04
8    1183493.83
9    1185703.90
Name: Y, dtype: float64


In [23]:
#Eliminar duplicados
df.drop_duplicates(inplace=True)
#borrar columnas con datos redudantes o poco aportantes
df.drop(labels=["DIRECCION ENCASILLADA","EXPEDIENTE","FECHA_ACCIDENTES","NRO_RADICADO","COMUNA"],axis=1,inplace=True)

In [24]:
df.to_csv("datosLimpios.csv")
#guardamos los datos en una nueva varible
data=df.copy()
df

,AÑO,CBML,CLASE_ACCIDENTE,DIRECCION,DISEÑO,FECHA_ACCIDENTE,GRAVEDAD_ACCIDENTE,MES,NUMCOMUNA,BARRIO,LOCATION,X,Y
0,2015,1013,caida ocupante,CR 46 CL 43,tramo de via,2015-10-21 05:58:00,con heridos,10,10,barrio colon,"[-75.5688011014, 6.24312304123]",834949.69,1182357.34
1,2015,1020,choque,CR 43 A CL 29,tramo de via,2015-11-05 03:00:00,solo daños,11,10,san diego,"[-75.5693883283, 6.22870030622]",834880.17,1180762.02
2,2015,0205,otro,CR 46 CL 37,tramo de via,2015-10-21 07:40:00,con heridos,10,2,la francia,"[-75.5503911403, 6.29869502502]",837004.94,1188499.17
3,2015,0101,atropello,CR 34 CL 112,tramo de via,2015-10-21 12:00:00,con heridos,10,1,santo domingo savio no. 1,"[-75.5442195069, 6.30019383306]",837688.53,1188663.05
4,2015,1504,choque,CR 50 CL 14,tramo de via,2015-11-03 06:16:00,solo daños,11,15,santa fe,"[-75.578180998, 6.21699243701]",833903.11,1179469.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270760,2020,1004,caida ocupante,CR 62 CL 59,tramo de via,2020-08-31 18:50:00,con heridos,8,10,el chagualo,"[-75.5731343044, 6.26063534822]",834475.51,1184295.99
270761,2020,1115,otro,CL 50 CR 74,tramo de via,2020-08-31 18:50:00,con heridos,8,11,cuarta brigada,"[-75.5902062951, 6.26040455473]",832585.61,1184275.87
270762,2020,515,choque,CL 80 CR 65,tramo de via,2020-08-31 19:05:00,solo daños,8,5,cementerio universal,"[-75.5735398111, 6.27625838886]",834435.55,1186024.41
270763,2020,207,atropello,CL 104 CR 45,tramo de via,2020-08-31 19:30:00,con heridos,8,2,villa del socorro,"[-75.5515085985, 6.29359892373]",836879.66,1187935.77


In [25]:
#df=pd.read_csv("datosLimpios.csv")

## 1 - Entrenamiento de un modelo predictivo
Se deberá construir un modelo que permita predecir la accidentalidad por tipo de accidente a nivel semanal, mensual y diario. Para esto se deberán considerar fechas especiales.

In [26]:
df=data
df["DIA"]=df["FECHA_ACCIDENTE"].apply(lambda x: x.date())
df=df.groupby(["DIA","CLASE_ACCIDENTE","NUMCOMUNA"]).count()
df.reset_index(inplace=True)
df["CANTIDAD"]=df["AÑO"]
df=df[["DIA","CLASE_ACCIDENTE","CANTIDAD","NUMCOMUNA"]]
df

,DIA,CLASE_ACCIDENTE,CANTIDAD,NUMCOMUNA
0,2014-07-04,choque,8,10
1,2014-07-04,choque,4,11
2,2014-07-04,choque,1,12
3,2014-07-04,choque,2,14
4,2014-07-04,choque,2,15
...,...,...,...,...
85313,2020-08-31,volcamiento,2,14
85314,2020-08-31,volcamiento,1,15
85315,2020-08-31,volcamiento,1,16
85316,2020-08-31,volcamiento,1,2


In [27]:
import holidays_co as co
from datetime import timedelta
#dia_festivo 1 si 0 no
df["dia_festivo".upper()]=df["DIA"].apply(lambda x : str(co.is_holiday_date(x)))
#sigiente_dia_festivo
#df["sigiente_dia_festivo".upper()]=df["DIA"].apply(lambda x : int(co.is_holiday_date(x+timedelta(days=1))))
#anterior_dia_festivo
#df["anterior_dia_festivo".upper()]=df["DIA"].apply(lambda x : int(co.is_holiday_date(x-timedelta(days=1))))
df["AÑO"]=df["DIA"].apply(lambda x : pd.Timestamp(x).year-2014)
df["t"]=df["DIA"].apply(lambda x : (pd.Timestamp(x).year-2014)*12+pd.Timestamp(x).month)
df["dia_de_la_semana".upper()]=df["dia".upper()].apply(lambda x : str(pd.Timestamp(x).day_of_week))
min=df["DIA"].min()
#df["dia_total".upper()]=df["dia".upper()].apply(lambda x : (x-min).days+1)
df["MES"]=df["DIA"].apply(lambda x : str(pd.Timestamp(x).month))
df["dia_del_mes".upper()]=df["dia".upper()].apply(lambda x :str( pd.Timestamp(x).day))

dfaux=df.copy()
df.head(10)

,DIA,CLASE_ACCIDENTE,CANTIDAD,NUMCOMUNA,DIA_FESTIVO,AÑO,t,DIA_DE_LA_SEMANA,MES,DIA_DEL_MES
0,2014-07-04,choque,8,10,False,0,7,4,7,4
1,2014-07-04,choque,4,11,False,0,7,4,7,4
2,2014-07-04,choque,1,12,False,0,7,4,7,4
3,2014-07-04,choque,2,14,False,0,7,4,7,4
4,2014-07-04,choque,2,15,False,0,7,4,7,4
5,2014-07-04,choque,2,16,False,0,7,4,7,4
6,2014-07-04,choque,3,7,False,0,7,4,7,4
7,2014-07-05,atropello,2,1,False,0,7,5,7,5
8,2014-07-05,atropello,3,10,False,0,7,5,7,5
9,2014-07-05,atropello,2,12,False,0,7,5,7,5


Sea X el con junto de parametros 

### Entrenamiento y validación
Los modelos predictivos se deberán construir con los datos de los años 2014, 2015, 2016, 2018. Se usan los accidentes del año 2019 y 2020 (¡alerta!) para validar los modelos.
Se deberá analizar el comportamiento del modelo predictivo en los años 2021 y 2022 y explicar las posibles desviaciones.

In [28]:
dfaux=df.copy()
#dfaux=pd.concat([dfaux,pd.get_dummies(dfaux.pop("NUMCOMUNA"),drop_first=True)], axis=1)
train=dfaux[(dfaux["AÑO"]) <5].drop(["DIA"],axis=1).drop(["AÑO"],axis=1)
test=dfaux.drop(train.index).drop(["DIA"],axis=1).drop(["AÑO"],axis=1)
X_train=train.copy()
Y_train=X_train.pop("CANTIDAD")
X_test=test.copy()
Y_test=X_test.pop("CANTIDAD")
print(sorted(X_train.NUMCOMUNA.unique()))
print(sorted(X_test.NUMCOMUNA.unique()))

['1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5', '6', '7', '8', '9']
['1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5', '6', '7', '8', '9']


In [29]:
X_train=pd.get_dummies(X_train,drop_first=True)
X_test=pd.get_dummies(X_test,drop_first=True)

In [30]:
train_stats = train.describe()
train_stats.pop("CANTIDAD")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
t,63232.0,33.177584,15.472609,7.0,20.0,33.0,46.0,60.0


In [31]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = (X_train)
normed_test_data = (Y_test)

In [32]:
def build_model():
    model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(X_train.keys())]),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    optimizer2 = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_absolute_percentage_error',
                optimizer=optimizer2,
                metrics=['mae', 'mse',"mean_absolute_percentage_error"])
    return model

In [33]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                4480      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________


In [34]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

array([[-0.07936272],
       [ 0.02508536],
       [-0.24816054],
       [-0.24271159],
       [-0.09373513],
       [-0.11221546],
       [-0.16635862],
       [-0.26034796],
       [-0.2274988 ],
       [-0.3766362 ]], dtype=float32)

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 50 == 0: print("\n","epoca",epoch,logs)
    print('.', end='')

EPOCHS = 500

history = model.fit(
  normed_train_data, Y_train,
  epochs=EPOCHS, validation_data=(X_test,Y_test))

Epoch 1/500
1976/1976 [==============================] - 7s 3ms/step - loss: 40.1457 - mae: 1.9617 - mse: 17.3850 - mean_absolute_percentage_error: 40.1457 - val_loss: 32.0965 - val_mae: 1.6803 - val_mse: 14.4747 - val_mean_absolute_percentage_error: 32.0965
Epoch 2/500
1976/1976 [==============================] - 6s 3ms/step - loss: 32.1444 - mae: 1.7091 - mse: 14.4606 - mean_absolute_percentage_error: 32.1444 - val_loss: 31.2825 - val_mae: 1.3288 - val_mse: 9.4090 - val_mean_absolute_percentage_error: 31.2825
Epoch 3/500
1976/1976 [==============================] - 6s 3ms/step - loss: 30.3179 - mae: 1.5193 - mse: 11.2754 - mean_absolute_percentage_error: 30.3179 - val_loss: 29.8814 - val_mae: 1.3510 - val_mse: 9.1902 - val_mean_absolute_percentage_error: 29.8814
Epoch 4/500
1976/1976 [==============================] - 6s 3ms/step - loss: 29.4409 - mae: 1.4083 - mse: 9.3439 - mean_absolute_percentage_error: 29.4409 - val_loss: 29.2071 - val_mae: 1.3428 - val_mse: 8.5608 - val_mean_abs

Epoch 33/500
1976/1976 [==============================] - 6s 3ms/step - loss: 27.3057 - mae: 1.1714 - mse: 5.5877 - mean_absolute_percentage_error: 27.3057 - val_loss: 29.4323 - val_mae: 1.3439 - val_mse: 7.9029 - val_mean_absolute_percentage_error: 29.4323
Epoch 34/500
1976/1976 [==============================] - 6s 3ms/step - loss: 27.3793 - mae: 1.1819 - mse: 5.6912 - mean_absolute_percentage_error: 27.3793 - val_loss: 29.1869 - val_mae: 1.3366 - val_mse: 8.2182 - val_mean_absolute_percentage_error: 29.1869
Epoch 35/500
1976/1976 [==============================] - 6s 3ms/step - loss: 27.2534 - mae: 1.1677 - mse: 5.5380 - mean_absolute_percentage_error: 27.2534 - val_loss: 29.1032 - val_mae: 1.2499 - val_mse: 6.7902 - val_mean_absolute_percentage_error: 29.1032
Epoch 36/500
1976/1976 [==============================] - 6s 3ms/step - loss: 27.3144 - mae: 1.1777 - mse: 5.6628 - mean_absolute_percentage_error: 27.3144 - val_loss: 29.1747 - val_mae: 1.3103 - val_mse: 7.8750 - val_mean_abs

Epoch 65/500
1976/1976 [==============================] - 6s 3ms/step - loss: 27.0323 - mae: 1.1445 - mse: 5.1778 - mean_absolute_percentage_error: 27.0323 - val_loss: 29.6035 - val_mae: 1.4400 - val_mse: 9.9229 - val_mean_absolute_percentage_error: 29.6035
Epoch 66/500
1976/1976 [==============================] - 6s 3ms/step - loss: 27.0343 - mae: 1.1471 - mse: 5.1554 - mean_absolute_percentage_error: 27.0343 - val_loss: 29.3070 - val_mae: 1.3467 - val_mse: 8.3364 - val_mean_absolute_percentage_error: 29.3070
Epoch 67/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.9386 - mae: 1.1390 - mse: 5.1241 - mean_absolute_percentage_error: 26.9386 - val_loss: 30.0492 - val_mae: 1.3381 - val_mse: 8.7044 - val_mean_absolute_percentage_error: 30.0492
Epoch 68/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.9167 - mae: 1.1321 - mse: 5.0437 - mean_absolute_percentage_error: 26.9167 - val_loss: 29.2800 - val_mae: 1.3578 - val_mse: 8.4241 - val_mean_abs

Epoch 97/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.7466 - mae: 1.1156 - mse: 4.7483 - mean_absolute_percentage_error: 26.7466 - val_loss: 29.3158 - val_mae: 1.2805 - val_mse: 7.5539 - val_mean_absolute_percentage_error: 29.3158
Epoch 98/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.7069 - mae: 1.1088 - mse: 4.7037 - mean_absolute_percentage_error: 26.7069 - val_loss: 29.7455 - val_mae: 1.4424 - val_mse: 9.8819 - val_mean_absolute_percentage_error: 29.7455
Epoch 99/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.7261 - mae: 1.1165 - mse: 4.7805 - mean_absolute_percentage_error: 26.7261 - val_loss: 29.4007 - val_mae: 1.4032 - val_mse: 9.2788 - val_mean_absolute_percentage_error: 29.4007
Epoch 100/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.7227 - mae: 1.1188 - mse: 4.7823 - mean_absolute_percentage_error: 26.7227 - val_loss: 29.5842 - val_mae: 1.3860 - val_mse: 8.7985 - val_mean_ab

Epoch 129/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.5578 - mae: 1.0954 - mse: 4.4883 - mean_absolute_percentage_error: 26.5578 - val_loss: 29.7334 - val_mae: 1.4501 - val_mse: 10.1015 - val_mean_absolute_percentage_error: 29.7334
Epoch 130/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.5507 - mae: 1.0978 - mse: 4.5105 - mean_absolute_percentage_error: 26.5507 - val_loss: 29.4420 - val_mae: 1.4146 - val_mse: 9.5947 - val_mean_absolute_percentage_error: 29.4420
Epoch 131/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.5949 - mae: 1.0997 - mse: 4.5693 - mean_absolute_percentage_error: 26.5949 - val_loss: 29.4109 - val_mae: 1.3865 - val_mse: 9.0009 - val_mean_absolute_percentage_error: 29.4109
Epoch 132/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.4813 - mae: 1.0942 - mse: 4.5411 - mean_absolute_percentage_error: 26.4813 - val_loss: 29.6238 - val_mae: 1.4334 - val_mse: 9.8617 - val_mea

...

Epoch 256/500
1976/1976 [==============================] - 5s 3ms/step - loss: 26.3083 - mae: 1.0820 - mse: 4.3758 - mean_absolute_percentage_error: 26.3083 - val_loss: 29.2423 - val_mae: 1.3639 - val_mse: 9.1716 - val_mean_absolute_percentage_error: 29.2423
Epoch 257/500
1976/1976 [==============================] - 5s 3ms/step - loss: 26.2986 - mae: 1.0791 - mse: 4.3476 - mean_absolute_percentage_error: 26.2986 - val_loss: 29.2628 - val_mae: 1.3773 - val_mse: 9.2210 - val_mean_absolute_percentage_error: 29.2628
Epoch 258/500
1976/1976 [==============================] - 5s 3ms/step - loss: 26.3790 - mae: 1.0839 - mse: 4.3577 - mean_absolute_percentage_error: 26.3790 - val_loss: 29.4343 - val_mae: 1.4339 - val_mse: 10.3035 - val_mean_absolute_percentage_error: 29.4343
Epoch 259/500
1976/1976 [==============================] - 5s 3ms/step - loss: 26.2955 - mae: 1.0785 - mse: 4.3241 - mean_absolute_percentage_error: 26.2955 - val_loss: 29.2889 - val_mae: 1.4033 - val_mse: 9.7040 - val_mea

1976/1976 [==============================] - 6s 3ms/step - loss: 26.2881 - mae: 1.0768 - mse: 4.2908 - mean_absolute_percentage_error: 26.2881 - val_loss: 29.5963 - val_mae: 1.4469 - val_mse: 10.4296 - val_mean_absolute_percentage_error: 29.5963
Epoch 288/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.2247 - mae: 1.0768 - mse: 4.2953 - mean_absolute_percentage_error: 26.2247 - val_loss: 29.3083 - val_mae: 1.4145 - val_mse: 10.1803 - val_mean_absolute_percentage_error: 29.3083
Epoch 289/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.2516 - mae: 1.0780 - mse: 4.2994 - mean_absolute_percentage_error: 26.2516 - val_loss: 29.6920 - val_mae: 1.4400 - val_mse: 10.0841 - val_mean_absolute_percentage_error: 29.6920
Epoch 290/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.2951 - mae: 1.0823 - mse: 4.3248 - mean_absolute_percentage_error: 26.2951 - val_loss: 29.3192 - val_mae: 1.3996 - val_mse: 9.5773 - val_mean_absolute_p

Epoch 319/500
1976/1976 [==============================] - 5s 3ms/step - loss: 26.2328 - mae: 1.0748 - mse: 4.2446 - mean_absolute_percentage_error: 26.2328 - val_loss: 29.7958 - val_mae: 1.4729 - val_mse: 10.9764 - val_mean_absolute_percentage_error: 29.7958
Epoch 320/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.2435 - mae: 1.0783 - mse: 4.2974 - mean_absolute_percentage_error: 26.2435 - val_loss: 29.8171 - val_mae: 1.4733 - val_mse: 10.7920 - val_mean_absolute_percentage_error: 29.8171
Epoch 321/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.2480 - mae: 1.0829 - mse: 4.4081 - mean_absolute_percentage_error: 26.2480 - val_loss: 29.3079 - val_mae: 1.3729 - val_mse: 9.1985 - val_mean_absolute_percentage_error: 29.3079
Epoch 322/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.2418 - mae: 1.0790 - mse: 4.3794 - mean_absolute_percentage_error: 26.2418 - val_loss: 29.8232 - val_mae: 1.4611 - val_mse: 10.5885 - val_m

1976/1976 [==============================] - 5s 3ms/step - loss: 26.1817 - mae: 1.0734 - mse: 4.2751 - mean_absolute_percentage_error: 26.1817 - val_loss: 29.7515 - val_mae: 1.4476 - val_mse: 10.2762 - val_mean_absolute_percentage_error: 29.7515
Epoch 351/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.1629 - mae: 1.0687 - mse: 4.1983 - mean_absolute_percentage_error: 26.1629 - val_loss: 29.8213 - val_mae: 1.4796 - val_mse: 11.1077 - val_mean_absolute_percentage_error: 29.8213
Epoch 352/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.2079 - mae: 1.0777 - mse: 4.3173 - mean_absolute_percentage_error: 26.2079 - val_loss: 29.2476 - val_mae: 1.3747 - val_mse: 9.2008 - val_mean_absolute_percentage_error: 29.2476
Epoch 353/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.2619 - mae: 1.0755 - mse: 4.2659 - mean_absolute_percentage_error: 26.2619 - val_loss: 29.6379 - val_mae: 1.4684 - val_mse: 10.9662 - val_mean_absolute_p

1976/1976 [==============================] - 6s 3ms/step - loss: 26.2363 - mae: 1.0726 - mse: 4.2623 - mean_absolute_percentage_error: 26.2363 - val_loss: 29.3790 - val_mae: 1.4235 - val_mse: 9.9280 - val_mean_absolute_percentage_error: 29.3790
Epoch 382/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.1725 - mae: 1.0671 - mse: 4.1418 - mean_absolute_percentage_error: 26.1725 - val_loss: 29.3020 - val_mae: 1.4048 - val_mse: 9.9628 - val_mean_absolute_percentage_error: 29.3020
Epoch 383/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.1529 - mae: 1.0692 - mse: 4.1930 - mean_absolute_percentage_error: 26.1529 - val_loss: 29.9275 - val_mae: 1.4308 - val_mse: 10.2326 - val_mean_absolute_percentage_error: 29.9275
Epoch 384/500
1976/1976 [==============================] - 6s 3ms/step - loss: 26.1921 - mae: 1.0699 - mse: 4.2027 - mean_absolute_percentage_error: 26.1921 - val_loss: 29.7450 - val_mae: 1.4692 - val_mse: 10.8934 - val_mean_absolute_pe

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()
hist.sort_values(by=['mean_absolute_percentage_error', 'val_mean_absolute_percentage_error']).head()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# Evalúe el modelo con los conjuntos de entrenamiento y prueba.
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(X_test)

    # Compute el error cuadratico medio de entrenamiento y prueba. Redondee los
    # valores a dos decimales.

mse_train = r2_score(
        Y_train,
        Y_train_pred,
    ).round(2)

mse_test = r2_score(
        Y_test,
        Y_test_pred,
    ).round(2)

In [ ]:
print(mse_test,mse_train)

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  #plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  #plt.ylim([0,20])
  plt.legend()
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('mean_absolute_percentage_error')
  plt.plot(hist['epoch'], hist['mean_absolute_percentage_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_percentage_error'],
           label = 'Val Error')
  #plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error,median_absolute_error
mae_train = median_absolute_error(
        Y_train,
        Y_train_pred,
    ).round(2)

mae_test = median_absolute_error(
        Y_test,
        Y_test_pred,
    ).round(2)

In [ ]:
print("entrenamiento:",mae_train,"validacion:",mae_test)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error,median_absolute_error
mape_train = mean_absolute_percentage_error(
        Y_train,
        Y_train_pred,
    ).round(2)

mape_test = mean_absolute_percentage_error(
        Y_test,
        Y_test_pred,
    ).round(2)

In [ ]:
print("entrenamiento:",mape_train,"validacion:",mape_test)

In [ ]:
resultTest=pd.DataFrame()
resultTest["esperado"]=Y_train
resultTest["predicho"]=Y_train_pred

In [ ]:
resultTest.sample(10)

In [ ]:
#model.save('modelo-V1.h5')
#optimarzer sin tener en cuenta el año

In [ ]:
#t=year
#model.save('modelo-V2.h5')
#optimarzer teniendo en cuenta el año 250 epocas
#entrenamiento: 0.19 validacion: 0.26
#model.save('modelo-V2_1.h5')
#entrenamiento: 0.17 validacion: 0.28
# 500 epocas

In [ ]:
from tensorflow.keras.models import load_model
#t=mounths
#250
#entrenamiento: 0.19 validacion: 0.15
#model.save('modelo-V3.h5')
#500
#entrenamiento: 0.17 validacion: 0.18
#model.save('modelo-V3_1.h5')
#model=load_model('modelo-V3.h5')